## Setup

In [1]:

#
import pandas as pd
import numpy as np


# make sql database connection with pyodbc
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde_tabular;UID=sde;PWD=staff')


SyntaxError: invalid syntax (1196091081.py, line 5)

# Air Quality

## Noise

In [ ]:
pip install geopandas

#### CNEL Maximum Day Update SDE.Monitoring SDE.Plan_Area_Noise

In [35]:
# Setup
#import pandas as pd
#import os
#from arcgis import GIS
#from arcgis.features import FeatureLayer
# Import geopandas
#import geopandas as gpd
#from shapely.geometry import Point
#import arcpy
#from arcgis.features import GeoAccessor


#workspace =r'C:\Users\snewsome\Documents\ArcGIS\Projects\Data Management 2023\Scratch.gdb'

## Update SDE.Monitoring.SDE.Plan_Area_Noise with new data 2020-2023
#excel_file_path = r"F:\Research and Analysis\Noise\Monitoring\CNEL - Plan Areas\ALL_CNEL_PLANAREAS_RESULTS_(1991-2023).xlsx"

# Read the Excel file into a DataFrame
#Noise23_df = pd.read_excel(excel_file_path)

# Keep only columns that you want
#columns_to_keep = ['CNEL LAND USE', '1987 PAS#', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'PAS_NAME', 'LAND_USE', '2023 CNEL Maximum Day', '2022 CNEL Maximum Day', '2021 CNEL Maximum Day', '2020 CNEL Maximum Day']
#Noise23_df = Noise23_df[columns_to_keep]

#Noise23_df.columns = Noise23_df.columns.str.strip()

# PASNAME
#Noise23_df['PAS_NAME'] = Noise23_df['PAS_NAME'].astype(str)

# Assuming df is your DataFrame
#meltcolumns_to_keep = ['CNEL LAND USE', '1987 PAS#', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'LAND_USE']
#id_columns = ['PAS_NAME']

# Specify the columns to melt (exclude the identifier column)
#columns_to_melt = ['2023 CNEL Maximum Day', '2022 CNEL Maximum Day', '2021 CNEL Maximum Day', '2020 CNEL Maximum Day']

#id_vars = id_columns + meltcolumns_to_keep

# Use pd.melt to reshape the DataFrame
#Noise23_df = pd.melt(Noise23_df, id_vars=id_vars, value_vars=columns_to_melt, var_name='Year', value_name='CNEL_Maximum_Day_Value')

#print(Noise23_df.head())
#print(Noise23_df.columns)

# Extract only the year from the 'YEAR_OF_COUNT' column and drop "CNEL Maximum Day" from each value
#Noise23_df['Year'] = Noise23_df['Year'].str.replace(' CNEL Maximum Day', '')



# Add 'Category' column
#Noise23_df['Category'] = 'CNEL Maximum Day'

# Drop rows where 'CNEL_Maximum_Day_Value' is NaN
#Noise23_df = Noise23_df.dropna(subset=['CNEL_Maximum_Day_Value'])

# Use lookup_dict to fill in LTINFO# Create a lookup dictionary to fill in LTINFO and ID
#gis = GIS()


#def get_fs_data(service_url):
 #   feature_layer = FeatureLayer(service_url)
  #  query_result = feature_layer.query()
   # feature_list = query_result.features
    #all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    #return all_data


# REST SERVICE data for lookup dictionary
#service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/6'

#dfplannoisesde = get_fs_data(service_url)

#dfplannoisesde.info()
#columnstokeep = ['SITE_NAME', 'ID', 'LTINFO']
#dfplannoisesde = dfplannoisesde.loc[:, columnstokeep]

# Drop duplicates based on 'SITE_NAME' and keep the first occurrence
#unique_values = dfplannoisesde.drop_duplicates(subset='SITE_NAME', keep='first')

# Select specific columns for lookup
#selected_columns = ['SITE_NAME', 'ID', 'LTINFO']

# Convert selected columns to dictionary
#lookup_dict = unique_values[selected_columns].set_index('SITE_NAME').to_dict(orient='index')

# Display the dictionary
#print(lookup_dict)

# Create new columns 'ID' and 'LTINFO' based on 'PAS_NAME' and look up dictionary
#Noise23_df['ID'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('ID'))
#Noise23_df['LTINFO'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('LTINFO'))

# Specify the columns to keep in Final_sdf
#finalcolumns_to_keep = ['PAS_NAME', 'Year', 'CNEL_Maximum_Day_Value', 'Category','ID', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'LAND_USE', 'CNEL LAND USE', 'LTINFO', '1987 PAS#']

# Create a new DataFrame Final_df with the selected columns
#Final_sdf = Noise23_df[finalcolumns_to_keep]

# Create a GeoDataFrame by converting 'LATITUDE' and 'LONGITUDE' to Point geometry
#geometry = [Point(xy) for xy in zip(Final_sdf['MONITORING SITE LONGITUDE'], Final_sdf['MONITORING SITE LATITUDE'])]
#Final_sdf = gpd.GeoDataFrame(Final_sdf, geometry=geometry)

# Field mapping so that columns match
#field_mapping = {
 #   '1987 PAS#': 'PAS_1987',
  #  'ID': 'ID',
   # 'MONITORING SITE LATITUDE': 'LATITUDE',
    #'MONITORING SITE LONGITUDE': 'LONGITUDE',
    #'PAS_NAME': 'SITE_NAME',
    #'LAND_USE': 'LAND_USE',
    #'CNEL LAND USE': 'CNEL_LAND_USE',
    #'Category': 'CATEGORY',
    #'Year': 'YEAR_OF_COUNT',
    #'CNEL_Maximum_Day_Value': 'COUNT_VALUE',
    #'LTINFO': 'LTINFO'
#}

# Create a new DataFrame Final_df with columns based on the field mapping
#Final_sdf = Final_sdf.rename(columns=field_mapping)

# Convert GeoPandas DataFrame to ArcGIS Spatially Enabled DataFrame
#sedf = GeoAccessor.from_geodataframe(Final_sdf)

# Display the updated Final_df
#print(list(Final_sdf.columns))

# Save Final_df to a CSV file
#Final_sdf.to_csv(r'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Drafts\NoiseThreshold23.csv', index=False)

# Convert the SEDF to a feature class
#sedf.spatial.to_featureclass(location=os.path.join(workspace, 'PlanAreaNoise23staging'), 
 #                                                           sanitize_columns=False)



               PAS_NAME             CNEL LAND USE 1987 PAS#  \
0            MOUNT ROSE       WILDERNESS/ROADLESS        30   
1         BIJOU MEADOWS  URBAN OUTDOOR RECREATION       101   
2              LAKEVIEW  HIGH DENSITY RESIDENTIAL        37   
3  ROUNDHILL COMMERCIAL                INDUSTRIAL        71   
4         MOUNTAIN VIEW   LOW DENSITY RESIDENTIAL       132   

   MONITORING SITE LATITUDE  MONITORING SITE LONGITUDE LAND_USE  \
0                 39.303580                -119.929140     CONS   
1                 38.921959                -119.961552      REC   
2                 39.248356                -119.971536      RES   
3                 38.984234                -119.941212   COM/PS   
4                 38.879719                -120.039483      RES   

                    Year  CNEL_Maximum_Day_Value  
0  2023 CNEL Maximum Day                     NaN  
1  2023 CNEL Maximum Day                     NaN  
2  2023 CNEL Maximum Day                     NaN  
3  2023 CNEL M

Could not insert the row because of error message: The value type is incompatible with the field type. [PAS_1987]. Recheck your data.
Could not insert the row because of error message: The value type is incompatible with the field type. [PAS_1987]. Recheck your data.
Could not insert the row because of error message: The value type is incompatible with the field type. [PAS_1987]. Recheck your data.


'C:\\Users\\snewsome\\Documents\\ArcGIS\\Projects\\Data Management 2023\\Scratch.gdb\\PlanAreaNoise23staging'

In [86]:
#updated to make a feature class instead of a csv
# Setup
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor
import os


workspace =r'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'

# Update SDE.Monitoring.SDE.Plan_Area_Noise with new data 2020-2023
excel_file_path = r"F:\Research and Analysis\Noise\Monitoring\CNEL - Plan Areas\ALL_CNEL_PLANAREAS_RESULTS_(1991-2023).xlsx"

# Read the Excel file into a DataFrame
Noise23_df = pd.read_excel(excel_file_path)

# Keep only columns that you want
columns_to_keep = ['CNEL LAND USE', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'PAS_NAME', 'LAND_USE', '2023 CNEL Maximum Day', '2022 CNEL Maximum Day', '2021 CNEL Maximum Day', '2020 CNEL Maximum Day']
Noise23_df = Noise23_df[columns_to_keep]

Noise23_df.columns = Noise23_df.columns.str.strip()

# PASNAME
Noise23_df['PAS_NAME'] = Noise23_df['PAS_NAME'].astype(str)

# melted columns to keep
meltcolumns_to_keep = ['CNEL LAND USE', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'LAND_USE']
id_columns = ['PAS_NAME']

# Specify the columns to melt (exclude the identifier column)
columns_to_melt = ['2023 CNEL Maximum Day', '2022 CNEL Maximum Day', '2021 CNEL Maximum Day', '2020 CNEL Maximum Day']

id_vars = id_columns + meltcolumns_to_keep

# Use pd.melt to reshape the DataFrame
Noise23_df = pd.melt(Noise23_df, id_vars=id_vars, value_vars=columns_to_melt, var_name='Year', value_name='CNEL_Maximum_Day_Value')

print(Noise23_df.head())
print(Noise23_df.columns)

# Extract only the year from the 'YEAR_OF_COUNT' column and drop "CNEL Maximum Day" from each value
Noise23_df['Year'] = Noise23_df['Year'].str.replace(' CNEL Maximum Day', '')



# Add 'Category' column
Noise23_df['Category'] = 'CNEL Maximum Day'

# Drop rows where 'CNEL_Maximum_Day_Value' is NaN
Noise23_df = Noise23_df.dropna(subset=['CNEL_Maximum_Day_Value'])

# Use lookup_dict to fill in LTINFO# Create a lookup dictionary to fill in LTINFO and ID
gis = GIS()


def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    feature_list = query_result.features
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    return all_data


# REST SERVICE data for lookup dictionary
service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/6'

dfplannoisesde = get_fs_data(service_url)

dfplannoisesde.info()
columnstokeep = ['SITE_NAME', 'ID', 'LTINFO', 'PAS_1987', 'LATITUDE', 'LONGITUDE']
dfplannoisesde = dfplannoisesde.loc[:, columnstokeep]

# Drop duplicates based on 'SITE_NAME' and keep the first occurrence
unique_values = dfplannoisesde.drop_duplicates(subset='SITE_NAME', keep='first')

# Select specific columns for lookup
selected_columns = ['SITE_NAME', 'ID', 'LTINFO', 'PAS_1987', 'LATITUDE', 'LONGITUDE']

def update_lat_lon_from_lookup_dict(noise_data, lookup_dict):
    # Assuming 'SITE_NAME' is the common identifier between DataFrame and lookup dictionary
    for index, row in noise_data.iterrows():
        # Get the 'LATITUDE' and 'LONGITUDE' from the lookup dictionary using the 'SITE_NAME' from the DataFrame
        lat_lon = lookup_dict.get(row['PAS_NAME'])
        # Update the 'MONITORING SITE LATITUDE' and 'MONITORING SITE LONGITUDE' in the DataFrame with the 'LATITUDE' and 'LONGITUDE' from the lookup dictionary
        if lat_lon is not None:
            noise_data.at[index, 'MONITORING SITE LATITUDE'] = lat_lon['LATITUDE']
            noise_data.at[index, 'MONITORING SITE LONGITUDE'] = lat_lon['LONGITUDE']

    
    return noise_data

# Convert selected columns to dictionary
lookup_dict = unique_values[selected_columns].set_index('SITE_NAME').to_dict(orient='index')

# Display the dictionary
print(lookup_dict)

# Update 'MONITORING SITE LATITUDE' and 'MONITORING SITE LONGITUDE' in Noise23_df with data from the lookup dictionary
Noise23_df = update_lat_lon_from_lookup_dict(Noise23_df, lookup_dict)



# Create new columns 'ID' and 'LTINFO' based on 'PAS_NAME' and look up dictionary
Noise23_df['ID'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('ID'))
Noise23_df['LTINFO'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('LTINFO'))
Noise23_df['PAS_1987'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('PAS_1987'))

# Specify the columns to keep in Final_sdf
finalcolumns_to_keep = ['PAS_NAME', 'Year', 'CNEL_Maximum_Day_Value', 'Category','ID', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'LAND_USE', 'CNEL LAND USE', 'LTINFO', 'PAS_1987']

# Create a new DataFrame Final_df with the selected columns
df = Noise23_df[finalcolumns_to_keep]

# Field mapping so that columns match
field_mapping = {
    'PAS_1987': 'PAS_1987',
    'ID': 'ID',
    'MONITORING SITE LATITUDE': 'LATITUDE',
    'MONITORING SITE LONGITUDE': 'LONGITUDE',
    'PAS_NAME': 'SITE_NAME',
    'LAND_USE': 'LAND_USE',
    'CNEL LAND USE': 'CNEL_LAND_USE',
    'Category': 'CATEGORY',
    'Year': 'YEAR_OF_COUNT',
    'CNEL_Maximum_Day_Value': 'COUNT_VALUE',
    'LTINFO': 'LTINFO'
}

# Create a new DataFrame Final_df with columns based on the field mapping
df = df.rename(columns=field_mapping)

# Convert DataFrame to Spatially Enabled DataFrame
sedf = GeoAccessor.from_xy(df, x_column='LONGITUDE', y_column='LATITUDE')

# Convert the SEDF to a feature class without sanitizing columns
sedf.spatial.to_featureclass(location=os.path.join(workspace, 'PlanAreaNoise_23_Staging'), sanitize_columns=False)


               PAS_NAME             CNEL LAND USE  MONITORING SITE LATITUDE  \
0            MOUNT ROSE       WILDERNESS/ROADLESS                 39.303580   
1         BIJOU MEADOWS  URBAN OUTDOOR RECREATION                 38.921959   
2              LAKEVIEW  HIGH DENSITY RESIDENTIAL                 39.248356   
3  ROUNDHILL COMMERCIAL                INDUSTRIAL                 38.984234   
4         MOUNTAIN VIEW   LOW DENSITY RESIDENTIAL                 38.879719   

   MONITORING SITE LONGITUDE LAND_USE                   Year  \
0                -119.929140     CONS  2023 CNEL Maximum Day   
1                -119.961552      REC  2023 CNEL Maximum Day   
2                -119.971536      RES  2023 CNEL Maximum Day   
3                -119.941212   COM/PS  2023 CNEL Maximum Day   
4                -120.039483      RES  2023 CNEL Maximum Day   

   CNEL_Maximum_Day_Value  
0                     NaN  
1                     NaN  
2                     NaN  
3                     NaN  


'F:\\Research and Analysis\\Workspace\\Sarah\\Data Management 2023\\Scratch.gdb\\PlanAreaNoise_23_Staging'

In [87]:
#reproject feature class 4236 to 26910
# Set the input and output paths
arcpy.env.workspace=r'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'
input_feature_class = 'PlanAreaNoise_23_Staging'
output_feature_class = 'PlanAreaNoise_23_ready'
 
# Specify the target coordinate system (EPSG code 26910 for UTM Zone 10N)
target_coordinate_system = arcpy.SpatialReference(26910)
 
# Use the Project tool to reproject the feature class
arcpy.management.Project(
    input_feature_class,
    output_feature_class,
    target_coordinate_system
)
 
print(f"Reprojection completed. Output feature class: {output_feature_class}")

Reprojection completed. Output feature class: PlanAreaNoise_23_ready


In [50]:
arcpy.Delete_management(r'c:\Users\snewsome\Documents\GitHub\ThresholdEvaluation\Scripts\2023\db_connect\ConnectionFile.sde')

<Result 'true'>

In [54]:
import arcpy
from time import strftime


#push data to sde
#Create database connection
inWorkspace = r"F:\Research and Analysis\Workspace\Sarah\SarahVector.sde"
arcpy.env.workspace = inWorkspace

# Specify the name of the new version and the parent version
new_version_name = "PlanAreaNoise_" + strftime("%Y-%m-%d")
parent_version = "sde.DEFAULT"
version_name_full = '"TAHOE\SNEWSOME".'+ new_version_name

# List all versions in the geodatabase
existing_versions = [version.name for version in arcpy.da.ListVersions(inWorkspace)]

RuntimeError: 

In [53]:
import arcpy
from time import strftime


#push data to sde
#Create database connection
inWorkspace=r"F:\Research and Analysis\Workspace\Sarah\SarahVector.sde"
arcpy.env.workspace=inWorkspace

# Specify the name of the new version and the parent version
new_version_name = "PlanAreaNoise_" + strftime("%Y-%m-%d")
parent_version = "sde.DEFAULT"
version_name_full = '"TAHOE\SNEWSOME".'+ new_version_name

# List all versions in the geodatabase
existing_versions = [version.name for version in arcpy.da.ListVersions(inWorkspace)]

# Check if the specified version exists
if version_name_full in existing_versions:
    # Delete the version
    arcpy.management.DeleteVersion(inWorkspace, version_name_full)
    print(f"Version '{version_name_full}' deleted successfully.")
else:
    print(f"Version '{version_name_full}' does not exist.")

# Create a new version
arcpy.CreateVersion_management(inWorkspace, parent_version, new_version_name, "PUBLIC")

# If you want to create a connection file, you can use the following code:
arcpy.CreateDatabaseConnection_management(
    out_folder_path='db_connect/',
    out_name="ConnectionFile.sde",
    database_platform="SQL_SERVER",  
    instance='sql12',
    database='sde',
    account_authentication="OPERATING_SYSTEM_AUTH",  
    version_type='TRANSACTIONAL',
    version=version_name_full
)

PlanNoisesde=os.path.join(version_name_full, "SDE.Monitoring\SDE.Plan_Area_Noise")
PlanAreaNoise = 'planareanoise'
arcpy.MakeFeatureLayer_management(PlanNoisesde, PlanAreaNoise)


# Create a new version
#arcpy.CreateVersion_management(inWorkspace, parent_version, new_version_name, "PUBLIC")
arcpy.ChangeVersion_management(PlanAreaNoise, 'TRANSACTIONAL', version_name_full, '')

#Start
edit = arcpy.da.Editor('db_connections/ConnectionFile.sde')
edit.startEditing(False, True)

# Append the records from the temporary feature class to the target feature class
arcpy.management.Append(
    inputs='PlanAreaNoise_23_ready',
    target=PlanNoisesde, 
    schema_type='NO_TEST'
)

#Stop
edit.stopEditing(True)



RuntimeError: 

In [45]:
try:
    arcpy.CreateVersion_management(inWorkspace, parent_version, new_version_name, "PUBLIC")
    print("Version created successfully.")
except arcpy.ExecuteError as e:
    print(f"Error: {e}")

Error: ERROR 001148: Cannot create a version with the specified name.
Failed to execute (CreateVersion).



In [ ]:
#Start
edit = arcpy.da.Editor('db_connections/ConnectionFile.sde')
edit.startEditing(False, True)

# Append the records from the temporary feature class to the target feature class
arcpy.management.Append(
    inputs='PlanAreaNoise_23_ready',
    target=os.path.join(version_full_name, "SDE.Monitoring\SDE.Plan_Area_Noise"), 
    schema_type='NO_TEST'
)

#Stop
edit.stopEditing(True)

In [110]:
arcpy.Delete_management("planareanoise")

<Result 'true'>

In [ ]:
# Start an edit session
edit = arcpy.da.Editor('db_connections/ConnectionFile.sde')
edit.startEditing(False, True)

    # Create a new version
arcpy.CreateVersion_management(inWorkspace, parent_version, new_version_name, "PUBLIC")

#### Shorezone Noise

                PAS_NAME  Year  CNEL_Maximum_Day_Value          Category  \
37         FISH HATCHERY  2023                    54.2  CNEL Maximum Day   
39  KINGSBURY COMMERCIAL  2023                    63.3  CNEL Maximum Day   
44      INDUSTRIAL TRACT  2023                    55.6  CNEL Maximum Day   
54         TRUCKEE MARSH  2023                    48.3  CNEL Maximum Day   
60             ECHO VIEW  2023                    44.2  CNEL Maximum Day   

                                               LTINFO  \
37  {'ID': 179.0, 'LTINFO': 'https://monitoring.la...   
39  {'ID': 140.0, 'LTINFO': 'https://monitoring.la...   
44  {'ID': 81.0, 'LTINFO': 'https://monitoring.lak...   
54  {'ID': nan, 'LTINFO': 'https://monitoring.lake...   
60                                               None   

                                                   ID  
37  {'ID': 179.0, 'LTINFO': 'https://monitoring.la...  
39  {'ID': 140.0, 'LTINFO': 'https://monitoring.la...  
44  {'ID': 81.0, 'LTINFO': 'http

## Stream/CSCI

In [56]:
#Calculate the Average CSCI score for TPAandTPBsites for 2020-2022 and fill in csv, Next code block will be splitting TPA and TPB into two different avg scores
#Setup
#Create Dictionary Usring Rest Service data

import pandas as pd
import os
from arcgis import GIS
from arcgis.features import FeatureLayer
import numpy as np
import requests
import arcpy

gis = GIS()
# Connect to TRPA Enterprise GIS Portal *if it's a service only shared with org
# portal_user = "TRPA_PORTAL_ADMIN"
# portal_pwd = str(os.environ.get('Password'))
# portal_url = "https://maps.trpa.org/portal/"
wk_memory = "memory" + "\\"

#Set up Workspace
# set workspace and sde connections 
working_folder = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries"
workspace      = "C:\GIS\Scratch.gdb"
arcpy.env.workspace = "C:\GIS\Scratch.gdb"

# network path to connection files
filePath = r"C:\\GIS\\DB_CONNECT"


## CSV to be updated with new average
StreamAvg = os.path.join(working_folder,"ThresholdData_Stream.csv")


def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features
    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    return all_data

# Service URL
service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/8'

# Get Stream data as a Spatially Enabled DataFrame
sdfStreamHab = get_fs_data(service_url)

# Keep only necessary columns
columnstokeep = ['SITE_NAME', 'STATION_TYPE', 'DURATION', 'LATITUDE', 'LONGITUDE', 'LTINFO', 'COUNT_VALUE', 'YEAR_OF_COUNT']
sdfStreamHab = sdfStreamHab.loc[:, columnstokeep]

# Query only 2020-2022 data
sdfStreamHab['Date'] = pd.to_datetime(sdfStreamHab['YEAR_OF_COUNT'], format='%Y')
filtered_df = sdfStreamHab[
    (sdfStreamHab['Date'].dt.year.between(2020, 2022)) & 
    (sdfStreamHab['SITE_NAME'].str.contains('TPB|TPA'))
].copy()

# Calculate the average of the 'COUNT_VALUE' column
AvgCSCI23 = filtered_df['COUNT_VALUE'].mean()

# Update CSV ThresholdData_Stream.csv
csv_path = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries\ThresholdData_Streams.csv"
existing_df = pd.read_csv(csv_path)

Description = "Average CSCI score of all trend sites (48 sites)"
Year = '2020-2022'
Value = AvgCSCI

existing_entry = existing_df[
    (existing_df['Description'] == Description) &
    (existing_df['Year'] == Year)
]

if existing_entry.empty:
    new_entry = {'Description': Description, 'Year': Year, 'Value': Value}
    existing_df = pd.concat([existing_df, pd.DataFrame([new_entry])], ignore_index=True)


existing_df.to_csv(csv_path, index=False)
print(f"CSV file '{csv_path}' has been updated.")



CSV file 'F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries\ThresholdData_Streams.csv' has been updated.


#### Avg Stream CSCI split into TPB and TPA for all years. in a new excel sheet

In [81]:
import pandas as pd
import os
from arcgis import GIS
from arcgis.features import FeatureLayer
import numpy as np

import arcpy

gis = GIS()

wk_memory = "memory" + "\\"

#Set up Workspace
# set workspace and sde connections 
working_folder = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries"
workspace      = "F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb"
arcpy.env.workspace = "F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb"



## excel to be updated with new averages split into TPA and TPB
StreamAvg = os.path.join(working_folder,"ThresholdData_StreamsTavg.xlsx")


def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features
    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    return all_data

# Service URL
service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/8'

# Get Stream data as REST SERVICE to a Spatially Enabled DataFrame
sdfStreamHab = get_fs_data(service_url)

# Keep only necessary columns
columnstokeep = ['SITE_NAME', 'STATION_TYPE', 'DURATION', 'LATITUDE', 'LONGITUDE', 'LTINFO', 'COUNT_VALUE', 'YEAR_OF_COUNT']
sdfStreamHab = sdfStreamHab.loc[:, columnstokeep]

#This is where the magic calculations happen for each threshold
# Mke year of count into date format
sdfStreamHab['Date'] = pd.to_datetime(sdfStreamHab['YEAR_OF_COUNT'], format='%Y')

# Define bins for grouping by year ranges
bins = [2010, 2012, 2014, 2016, 2020, 2023]

# Define labels for the bins
labels = ['2010-2011', '2012-2013', '2014-2015', '2016-2019', '2020-2022']

# Create a new column 'YearRange' to group the data by year ranges
sdfStreamHab['YearRange'] = pd.cut(sdfStreamHab['Date'].dt.year, bins=bins, labels=labels, right=False)

# Filter the DataFrame for TPB sites and TPA sites
filtered_df_tpb = sdfStreamHab[sdfStreamHab['SITE_NAME'].str.contains('TPB')]
filtered_df_tpa = sdfStreamHab[sdfStreamHab['SITE_NAME'].str.contains('TPA')]

# Group by 'YearRange', then calculate the average 'COUNT_VALUE' TPB sites
averages_tpb = filtered_df_tpb.groupby('YearRange')['COUNT_VALUE'].mean().reset_index()
averages_tpb.rename(columns={'COUNT_VALUE': 'Average_COUNT_VALUE_TPB'}, inplace=True)

averages_tpa = filtered_df_tpa.groupby('YearRange')['COUNT_VALUE'].mean().reset_index()
averages_tpa.rename(columns={'COUNT_VALUE': 'Average_COUNT_VALUE_TPA'}, inplace=True)

# Create Description and Year columns in both DataFrames
averages_tpb['Description'] = 'Average CSCI Score per Trend Panel(24 Sites)'
averages_tpa['Description'] = 'Average CSCI Score per Trend Panel(24 Sites)'
averages_tpb['Year'] = averages_tpb['YearRange']
averages_tpa['Year'] = averages_tpa['YearRange']

# Merge the two DataFrames on 'Year' and 'Description' to create the final merged DataFrame
FAdf = pd.merge(averages_tpb, averages_tpa, on=['Year', 'Description'], how='outer')
# Sort by 'Year' for clarity
FAdf.sort_values('Year', inplace=True)

# Reset index for clarity
FAdf.reset_index(drop=True, inplace=True)

# Define columns to keep
columns_to_keep = ['Year', 'Description', 'Average_COUNT_VALUE_TPB', 'Average_COUNT_VALUE_TPA']

# Select only the desired columns
FAdf = FAdf[columns_to_keep]
# Print the combined averages
print(FAdf)
print("FAdf:")

# Write the updated DataFrame back to the CSV file
FAdf.to_excel(StreamAvg, index=False)

print(f"The excel file '{StreamAvg}' has been updated with new averages successfully.")

        Year                                   Description  \
0  2010-2011  Average CSCI Score per Trend Panel(24 Sites)   
1  2012-2013  Average CSCI Score per Trend Panel(24 Sites)   
2  2014-2015  Average CSCI Score per Trend Panel(24 Sites)   
3  2016-2019  Average CSCI Score per Trend Panel(24 Sites)   
4  2020-2022  Average CSCI Score per Trend Panel(24 Sites)   

   Average_COUNT_VALUE_TPB  Average_COUNT_VALUE_TPA  
0                 0.987351                 0.944301  
1                 0.977152                 0.896196  
2                 0.972934                 0.918140  
3                 0.916064                 0.918429  
4                 0.890343                 1.105756  
FAdf:
The excel file 'F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries\ThresholdData_StreamsTavg.xlsx' has been updated with new averages successfully.


In [ ]:
#Use F:Research and Analysis/Threshold reporting\ThresholdData\Fisheries\ThresholdData_Streams.csv' to update SDE.tabular.....table


In [34]:
#Calculate Rating or CSCI values and then find the percent of total streams in each rating
excel_file_path="F:\Research and Analysis\Fisheries\Streams\Bioassessment\California Stream Condition Index\California Stream Condition Index\CSCI_Scores_AllSites_AllYears.xlsx"

Alldf = pd.read_excel(excel_file_path, sheet_name='CSCI_Scores_AllSites_AllYears')

#Calculate Rating for CSCI value
#Define a function to categorize CSCI values based on ranges
def categorize_value(value):
    if 0 <= value < 0.8:
        return 'marginal'
    elif 0.8 <= value < 1.0:
        return 'good'
    else:
        return 'excellent'

Alldf['Rating']=Alldf['CSCI_Score'].apply(categorize_value)

print(Alldf)

#Find average for each rating- Poor, Marginal, Good, Excellent
# Group by 'Category' and calculate the mean of 'Count'
average_count_by_category = df.groupby('Rating')['CSCI_Score'].mean()    

    StationCod StationCode  Sample Year  Organism Count  -120.01605  \
0    634UTR004   634UTR004         2000             NaN -120.001460   
1    634UTR005   634UTR005         2000             NaN -120.023080   
2    634UTR006   634UTR006         2000             NaN -120.020990   
3    634S13242   634S13242         2000             NaN -119.990880   
4    634UTR002   634UTR002         2000             NaN -119.991390   
..         ...         ...          ...             ...         ...   
598  634TPB134   634TPB134         2020             NaN -120.107720   
599  634TPB138   634TPB138         2020             NaN -119.917980   
600  634TPB143   634TPB143         2020             NaN -120.224850   
601  634TPB153   634TPB153         2020             NaN -120.036040   
602  634TRT002   634TRT002         2020             NaN -119.976433   

      38.76332  CSCI_Score     Rating           StationTyp  
0    38.879120    0.930000       good  Targeted Monitoring  
1    38.862350    0.99000

KeyError: 'Rating'